In [91]:
import sqlite3
from sqlite3 import Error
import os

path = os.getcwd()


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid


def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid


In [92]:
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

In [93]:
# create a database connection
conn = create_connection(path + "/pythonsqlite.db")

# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)

    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')
    task_3 = ('Confirm with user about the top requirements', 2, 2, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)
    create_task(conn, task_3)

else:
    print("Error! cannot create the database connection.")

In [94]:
cur = conn.cursor()
cur.execute("SELECT * FROM projects").fetchall()

[(1, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30')]

In [95]:
cur.execute("SELECT * FROM tasks").fetchone()

(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')

In [96]:
cur.close()

In [97]:
import pandas as pd

# Read sqlite query results into a pandas DataFrame
tasks_df = pd.read_sql_query("SELECT * from tasks", conn)

# Verify that result of SQL query is stored in the dataframe
tasks_df.head()


,id,name,priority,status_id,project_id,begin_date,end_date
0,1,Analyze the requirements of the app,1,1,1,2015-01-01,2015-01-02
1,2,Confirm with user about the top requirements,1,1,1,2015-01-03,2015-01-05
2,3,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05


In [98]:
# Select only data for status_id = 2
tasks_df = tasks_df[tasks_df.status_id == 2]

# Write the new DataFrame to a new SQLite table
tasks_df.to_sql("tasks", conn, if_exists="replace")


1

In [99]:
new_tasks_df = pd.read_sql_query("SELECT * from tasks", conn)
new_tasks_df

,index,id,name,priority,status_id,project_id,begin_date,end_date
0,2,3,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05


In [100]:
# !pip3 install faker
from faker import Faker
import pandas as pd
fake = Faker()
profileData = [fake.simple_profile() for i in range(100)]
profiles = pd.DataFrame(profileData)

profiles.head()

,username,name,sex,address,mail,birthdate
0,rcampbell,Jordan Murphy,M,Unit 3180 Box 5896\nDPO AA 50281,adamshailey@hotmail.com,1974-10-03
1,mmcdowell,Robyn Carter,F,"455 Gregory Center\nNew Troyland, AS 88150",joshuasmith@yahoo.com,1950-06-30
2,ashleyross,Samuel Chandler,M,"1105 Walker Bypass Suite 765\nPort Keith, ID 3...",bryannguyen@yahoo.com,1948-06-30
3,patrickthompson,Melissa Rodriguez,F,"7597 Barbara Plaza Suite 462\nWest Amanda, AZ ...",jmcdonald@gmail.com,1987-01-10
4,tflores,Ashley Brooks,F,"394 Decker Land Suite 164\nMendezborough, VA 5...",floresashley@hotmail.com,2014-02-16


In [101]:
profiles.columns.tolist()

['username', 'name', 'sex', 'address', 'mail', 'birthdate']

In [102]:
conn.execute('CREATE TABLE IF NOT EXISTS profiles (username, name, sex, address, mail, birthdate)')
conn.commit()
profiles.to_sql('profiles', conn, if_exists='replace', index = False)

profiles_df = pd.read_sql_query("SELECT * from profiles", conn)
profiles_df

,username,name,sex,address,mail,birthdate
0,rcampbell,Jordan Murphy,M,Unit 3180 Box 5896\nDPO AA 50281,adamshailey@hotmail.com,1974-10-03
1,mmcdowell,Robyn Carter,F,"455 Gregory Center\nNew Troyland, AS 88150",joshuasmith@yahoo.com,1950-06-30
2,ashleyross,Samuel Chandler,M,"1105 Walker Bypass Suite 765\nPort Keith, ID 3...",bryannguyen@yahoo.com,1948-06-30
3,patrickthompson,Melissa Rodriguez,F,"7597 Barbara Plaza Suite 462\nWest Amanda, AZ ...",jmcdonald@gmail.com,1987-01-10
4,tflores,Ashley Brooks,F,"394 Decker Land Suite 164\nMendezborough, VA 5...",floresashley@hotmail.com,2014-02-16
...,...,...,...,...,...,...
95,gibbspeter,Jon Wilkinson,M,"827 Kristen Inlet Suite 287\nWest Anthonytown,...",mary58@hotmail.com,1911-08-29
96,patrickwilliams,Jose Smith,M,"649 Brittney Trace\nNew Adamtown, AZ 87327",brianlopez@hotmail.com,1925-06-27
97,kevinaustin,Alison Gay DDS,F,USNV Chambers\nFPO AE 41839,rodriguezcheryl@yahoo.com,2016-11-02
98,joshuaramirez,Rebecca Strickland,F,"921 Christian Motorway\nNoahberg, GU 90720",haneytravis@hotmail.com,1917-12-25


In [103]:
import random

salaries = pd.DataFrame({"username":profiles.username, 
                         "salary": random.sample(range(100000,500000),len(profiles.username))})

salaries.head()

,username,salary
0,rcampbell,353964
1,mmcdowell,365774
2,ashleyross,471509
3,patrickthompson,219904
4,tflores,222689


In [104]:
salaries.columns.tolist()

['username', 'salary']

In [105]:
conn.execute('CREATE TABLE IF NOT EXISTS salaries (username, salary)')
conn.commit()
salaries.to_sql('salaries', conn, if_exists='replace', index = False)

salaries_df = pd.read_sql_query("SELECT * from salaries", conn)
salaries_df

,username,salary
0,rcampbell,353964
1,mmcdowell,365774
2,ashleyross,471509
3,patrickthompson,219904
4,tflores,222689
...,...,...
95,gibbspeter,204152
96,patrickwilliams,466662
97,kevinaustin,211594
98,joshuaramirez,190775


In [114]:
inner_join = pd.read_sql_query("SELECT * from salaries s inner join profiles p on p.username == s.username", conn)
inner_join

,username,salary,username,name,sex,address,mail,birthdate
0,rcampbell,353964,rcampbell,Jordan Murphy,M,Unit 3180 Box 5896\nDPO AA 50281,adamshailey@hotmail.com,1974-10-03
1,mmcdowell,365774,mmcdowell,Robyn Carter,F,"455 Gregory Center\nNew Troyland, AS 88150",joshuasmith@yahoo.com,1950-06-30
2,ashleyross,471509,ashleyross,Samuel Chandler,M,"1105 Walker Bypass Suite 765\nPort Keith, ID 3...",bryannguyen@yahoo.com,1948-06-30
3,patrickthompson,219904,patrickthompson,Melissa Rodriguez,F,"7597 Barbara Plaza Suite 462\nWest Amanda, AZ ...",jmcdonald@gmail.com,1987-01-10
4,tflores,222689,tflores,Ashley Brooks,F,"394 Decker Land Suite 164\nMendezborough, VA 5...",floresashley@hotmail.com,2014-02-16
...,...,...,...,...,...,...,...,...
95,gibbspeter,204152,gibbspeter,Jon Wilkinson,M,"827 Kristen Inlet Suite 287\nWest Anthonytown,...",mary58@hotmail.com,1911-08-29
96,patrickwilliams,466662,patrickwilliams,Jose Smith,M,"649 Brittney Trace\nNew Adamtown, AZ 87327",brianlopez@hotmail.com,1925-06-27
97,kevinaustin,211594,kevinaustin,Alison Gay DDS,F,USNV Chambers\nFPO AE 41839,rodriguezcheryl@yahoo.com,2016-11-02
98,joshuaramirez,190775,joshuaramirez,Rebecca Strickland,F,"921 Christian Motorway\nNoahberg, GU 90720",haneytravis@hotmail.com,1917-12-25
